```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/activation.ipynb>`, or a {download}`python script <converted/activation.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 4: Activation Maps

This tutorial will discuss how to compute local activation times and activation maps from cardiac optical mapping data using ``optimap``. Local activation times (often referred to as LATs) are times specified in frames or milliseconds at which the tissue became electrically activated. Computing local activation times corresponds to determining when the optical signal in a given pixel passes a certain pre-defined threshold or intensity value. For instance, if the optical trace is normalized and fluctuates betwen [0,1] and an action potential darkens the image (this is the convention that we use in Tutorials 1 and 2), then the tissue could be defined as being electrically 'activated' when the time-series goes below 0.5. We will discuss several examples in this tutorial.

First, we load and preprocess an example dataset in which a planar action potential wave propagates across the ventricles of a rabbit heart:

In [ ]:
import optimap as om
import numpy as np
import matplotlib.pyplot as plt

filename = om.utils.retrieve_example_data('Example_01_Sinus_Rabbit_Basler.npy')
video = om.load_video(filename)
video = om.video.rotate_left(video)
video=video[:20,:,:]
om.print_properties(video)
video_warped = om.motion.motion_compensate(video, 5, ref_frame=0)
video_warped_norm = om.video.normalize_pixelwise(video_warped)
om.print_properties(video_warped_norm)

Because the rabbit heart was stained with the voltage-sensitive dye Di-4-ANEPPS, the tissue becomes darkens when the tisue depolarizes:

In [ ]:
om.video.play2(video, video_warped_norm, title1="original video", title2="warped, normalized video", interval=100);

In [ ]:
render_ani_func(lambda: om.video.play2(video, video_warped_norm, title1="original video", title2="warped, normalized video", interval=20))

Let's plot some of the video frames as the wave propagates across the ventricles:

In [ ]:
figure, axarr = plt.subplots(1, 5)
axarr[0].imshow(video[0, :, :], cmap='gray')
axarr[1].imshow(video_warped_norm[1,:,:], cmap='gray', vmin=0, vmax=1)
axarr[2].imshow(video_warped_norm[6,:,:], cmap='gray', vmin=0, vmax=1)
axarr[3].imshow(video_warped_norm[10,:,:], cmap='gray', vmin=0, vmax=1)
axarr[4].imshow(video_warped_norm[14,:,:], cmap='gray', vmin=0, vmax=1)
figure.show()

And let's also plot some of the optical traces:

In [ ]:
positions = [(192, 167), (204, 141), (118, 158), (183, 267)]
traces = om.extract_traces(video_warped_norm, positions, size=3, show=True, fps=500)

We can now compute an activation map by identifying the timepoints in each pixel that correspond to when the action potential wave front passes through that pixel.

## Computing Activation Maps from Pixel-wise Normalized Optical Maps

The pixel-wise normalized video contains values between 0 and 1:

In [ ]:
om.print_properties(video_warped_norm)

Let's first invert the video:

In [ ]:
video_warped_norm = video_warped_norm*-1.0+1.0

Let's plot some of the optical traces (manually selected so that they show locations which become subsequently activated):

In [ ]:
positions =  [(227, 181), (199, 162), (213, 171), (240, 189), (176, 146), (188, 153)]
traces = om.extract_traces(video_warped_norm, positions, size=3, show=True, fps=500)

We can use ``optimap``'s {func}`compute_activation_map` function to automatically compute a two-dimensional activation map which shows the local activation times (here in frames because fps is unspecified):

In [ ]:
activation_map = om.activation.compute_activation_map(video_warped_norm, threshold=0.5, fps=None)

The range of local activation times can be displayed with:

In [ ]:
om.print_properties(activation_map)

Let's plot the activation map using 2 differrent colormaps:

In [ ]:
figure, axarr = plt.subplots(1, 3)
axarr[0].imshow(video[0, :, :], cmap='gray')
axarr[1].imshow(activation_map, cmap='hsv', vmin=0, vmax=20)
axarr[2].imshow(activation_map, cmap='jet', vmin=0, vmax=20)
figure.show()

You can see a continuois activation of the tissue.

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

In [ ]:
#video_diff = om.video.temporal_difference(video_warped, 5)
#video_diff[:, background_mask] = np.nan